<a href="https://colab.research.google.com/github/GenaroHacker/creating_chord_collection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Set Up
%%capture
!git clone https://github.com/GenaroHacker/creating_chord_collection.git
from creating_chord_collection.transposable_figures import transposable_figures
from creating_chord_collection.scales import scales

In [2]:
# @title Chord
class GuitarChord:
    all_notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    open_string_notes = ["E", "B", "G", "D", "A", "E"]

    def __init__(self, root, chord_type, transposable_figures, *, starting_fret=0, finger_ascending):
        self.root = root
        self.chord_type = chord_type
        self.starting_fret = starting_fret
        self.finger_ascending = finger_ascending
        self.transposable_figures = transposable_figures

    def __str__(self):
        return f"({repr(self.root)}, {repr(self.chord_type)}, finger_ascending={self.finger_ascending}, starting_fret={self.starting_fret})"

    def calculate_frequencies(self):
        frequencies = {}
        base_frequencies = {
            "E": [329.63, 82.41],  # 1st and 6th strings
            "B": [246.94],         # 2nd string
            "G": [196],            # 3rd string
            "D": [146.83],         # 4th string
            "A": [110]             # 5th string
        }

        def calculate_frequency(open_note, string_number, fret_position):
            base_frequency = base_frequencies[open_note][0] if string_number != 6 else base_frequencies[open_note][1]
            return base_frequency * (2 ** (fret_position / 12))

        for string_number, (open_note, finger_position) in enumerate(zip(GuitarChord.open_string_notes, self.finger_ascending), start=1):
            if finger_position is None:
                continue

            fret_position = self.starting_fret + finger_position - 1 if finger_position > 0 else 0
            frequency = calculate_frequency(open_note, string_number, fret_position)
            frequencies[string_number] = frequency

        return frequencies


    def get_notes(self, include_strings=False):
        def calculate_note(string, fret):
            if fret is None:
                return None
            if fret == 0:  # For open strings, return the default open string note
                return string
            # Calculate the note for fretted strings
            note_index = (GuitarChord.all_notes.index(string) + self.starting_fret + fret - 1) % len(GuitarChord.all_notes)
            return GuitarChord.all_notes[note_index]

        if include_strings:
            notes = {}
            for string_number, (string, fret) in enumerate(zip(GuitarChord.open_string_notes, self.finger_ascending), start=1):
                note = calculate_note(string, fret)
                notes[string_number] = note
            return notes
        else:
            frequencies = self.calculate_frequencies()
            notes = [calculate_note(string, fret) for string, fret in zip(GuitarChord.open_string_notes, self.finger_ascending) if fret is not None]
            unique_notes = list(dict.fromkeys(notes))  # Remove duplicates

            # Correctly map notes to string numbers for sorting
            note_to_string = {}
            for idx, (string, fret) in enumerate(zip(GuitarChord.open_string_notes, self.finger_ascending), start=1):
                if fret is not None:
                    note = calculate_note(string, fret)
                    note_to_string[note] = idx

            # Sort notes based on frequencies
            return sorted(unique_notes, key=lambda note: frequencies.get(note_to_string.get(note), float('inf')))



    def is_open(self):
        return 0 in self.finger_ascending

    def transpose(self, distance):
        # Helper function to transpose figure
        def transpose_figure(lst, num):
            return [item + num if item is not None else None for item in lst]

        # Helper function to raise specific errors after reverting changes
        def raise_transpose_error(error_type):
            self.root = original_root
            self.finger_ascending = original_finger_ascending
            self.starting_fret = original_starting_fret

            error_messages = {
                "below_0": "Chord transposition results in a note below the 0th fret.",
                "above_12": "Chord transposition results in a note above the 12th fret.",
                "not_equivalent_transposable_figure": "Chord figure is not equivalent to any figure in transposable_figures."
            }
            raise ValueError(error_messages[error_type])

        # Save original state for possible reversion
        original_root = self.root
        original_finger_ascending = self.finger_ascending.copy()
        original_starting_fret = self.starting_fret

        # Return if distance is zero
        if distance == 0:
            return

        # Update root note
        new_note_index = (GuitarChord.all_notes.index(self.root) + distance) % len(GuitarChord.all_notes)
        self.root = GuitarChord.all_notes[new_note_index]

        # Transpose open chords
        if self.is_open():
            if distance < 0:
                raise_transpose_error("below_0")
            elif distance > 0:
                self.finger_ascending = transpose_figure(self.finger_ascending, 1)
                self.starting_fret = max(0, self.starting_fret + distance - 1)
        else:  # Transpose barre chords
            if distance < 0:
                new_starting_fret = self.starting_fret + distance
                if new_starting_fret < 0:
                    raise_transpose_error("below_0")
                elif new_starting_fret == 0:
                    self.finger_ascending = transpose_figure(self.finger_ascending, -1)
                    self.starting_fret = 1  # Keeping the fret at 1
                else:
                    self.starting_fret = new_starting_fret
            else:  # Transpose barre chord to the right
                self.starting_fret += distance

        # Check for errors in transposition
        if any(fret < 0 for fret in self.finger_ascending if fret is not None):
            raise_transpose_error("below_0")
        if self.starting_fret > 9:
            raise_transpose_error("above_12")

        # Check transposability
        transposed_figure = self.finger_ascending if self.starting_fret == 0 else transpose_figure(self.finger_ascending, 1)
        if transposed_figure not in self.transposable_figures:
            raise_transpose_error("not_equivalent_transposable_figure")

    def validate_against_scale(self, tonic, scale):
        # Reorder the all_notes list starting from the tonic
        tonic_index = GuitarChord.all_notes.index(tonic)
        reordered_notes = GuitarChord.all_notes[tonic_index:] + GuitarChord.all_notes[:tonic_index]

        # Extract the notes of the scale using the provided scale pattern
        scale_notes = [reordered_notes[i] for i in scale]

        # Get the notes of the chord
        chord_notes = self.get_notes()

        # Check if all chord notes are in the scale
        return all(note in scale_notes for note in chord_notes)




# Example
chord = GuitarChord('E', 'm', transposable_figures, finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
print(chord)
print("String Frequencies:", chord.calculate_frequencies())
print("Is Open Chord:", chord.is_open())
chord.transpose(0)
print(chord.get_notes(include_strings=True))
print(chord.get_notes())

# Validate the chord against the C major scale
ionian_scale = scales["ionian"]
tonic = "C"
is_chord_in_scale = chord.validate_against_scale(tonic, ionian_scale)
print(f"Chord fits in C major scale: {is_chord_in_scale}")


('E', 'm', finger_ascending=[0, 0, 0, 2, 2, 0], starting_fret=1)
String Frequencies: {1: 329.63, 2: 246.94, 3: 196.0, 4: 164.81110255326524, 5: 123.47082531403103, 6: 82.41}
Is Open Chord: True
{1: 'E', 2: 'B', 3: 'G', 4: 'E', 5: 'B', 6: 'E'}
['E', 'B', 'G']
Chord fits in C major scale: True


In [43]:
# @title Collection
import sqlite3
from collections import defaultdict

class ChordCollection:
    def __init__(self):
        self.chords = []

    def load(self, db_path):
        self.chords.clear()
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()

        cursor.execute('SELECT ROOT, TYPE, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6 FROM TABLE_CHORDS')
        rows = cursor.fetchall()

        for row in rows:
            root, chord_type, starting_fret, *fingers = row
            chord = GuitarChord(root, chord_type, transposable_figures, finger_ascending=fingers, starting_fret=starting_fret)
            self.chords.append(chord)

        connection.close()

    def save(self, db_name):
        connection = sqlite3.connect(db_name)
        cursor = connection.cursor()

        # Create the table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TABLE_CHORDS (
                ID INTEGER PRIMARY KEY,
                ROOT TEXT,
                TYPE TEXT,
                STARTING_FRET INTEGER,
                STRING_1 INTEGER,
                STRING_2 INTEGER,
                STRING_3 INTEGER,
                STRING_4 INTEGER,
                STRING_5 INTEGER,
                STRING_6 INTEGER
            )
        ''')

        # Insert the chords
        for chord in self.chords:
            cursor.execute('''
                INSERT INTO TABLE_CHORDS (ROOT, TYPE, STARTING_FRET, STRING_1, STRING_2, STRING_3, STRING_4, STRING_5, STRING_6)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (chord.root, chord.chord_type, chord.starting_fret, *chord.finger_ascending))

        connection.commit()
        connection.close()

    def chord_exists(self, new_chord):
        for chord in self.chords:
            if chord.root == new_chord.root and chord.chord_type == new_chord.chord_type and chord.starting_fret == new_chord.starting_fret and chord.finger_ascending == new_chord.finger_ascending:
                return True
        return False

    def extend_barre_chords(self):
        original_chords = self.chords.copy()
        for chord in original_chords:
            counter = 1
            while True:
                try:
                    new_chord = GuitarChord(chord.root, chord.chord_type, chord.transposable_figures, finger_ascending=chord.finger_ascending.copy(), starting_fret=chord.starting_fret)
                    new_chord.transpose(counter)
                    if not self.chord_exists(new_chord):
                        self.chords.append(new_chord)
                    counter += 1
                except ValueError:
                    break

    def only(self, whitelist):
        # Define the helper functions for each filter criterion
        def filter_root(chord, values):
            return chord.root in values

        def filter_chord_type(chord, values):
            return chord.chord_type in values

        def filter_open(chord, values):
            return chord.is_open() in values

        def filter_starting_fret(chord, values):
            return chord.starting_fret in values

        def filter_include_string(chord, values):
            # Check if all strings in values are included (i.e., not None in finger_ascending)
            return all(chord.finger_ascending[string - 1] is not None for string in values)

        def filter_inversion(chord, values):
            notes = chord.get_notes(include_strings=False)

            # Determine the inversion based on the position of the root note
            if chord.root in notes:
                root_position = notes.index(chord.root)
                if root_position == 0:  # Root is the first note
                    inversion = 1
                elif root_position == 1:  # Root is the second note
                    inversion = 2
                else:  # Root is in any other position
                    inversion = 3
            else:
                inversion = None

            return inversion in values

        def filter_scale(chord, scales):
            for tonic, scale in scales:
                if chord.validate_against_scale(tonic, scale):
                    return True
            return False

        # Map each whitelist key to its corresponding filter function
        filter_functions = {
            "root": filter_root,
            "chord_type": filter_chord_type,
            "open": filter_open,
            "starting_fret": filter_starting_fret,
            "include_string": filter_include_string,
            "inversion": filter_inversion,
            "scale": filter_scale  # New entry for scale
        }

        # Filter the chords
        filtered_chords = []
        for chord in self.chords:
            if all(filter_functions[key](chord, values) for key, values in whitelist.items()):
                filtered_chords.append(chord)

        return filtered_chords

    def filter_out(self, blacklist):
        # Get chords that match the blacklist criteria
        matching_chords = self.only(blacklist)

        # Subtract matching chords from the original list
        remaining_chords = [chord for chord in self.chords if chord not in matching_chords]

        return remaining_chords




# Example usage
chord_collection = ChordCollection()
chord_collection.load('/content/creating_chord_collection/chord_collection.db')

chord = chord_collection.chords[6]
print(chord)
chord.transpose(1)
print(chord)
chord_collection.save('new_chord_collection.db')
print(f"Collection Size: {len(chord_collection.chords)}")
chord_collection.extend_barre_chords()
print(f"Collection Size: {len(chord_collection.chords)}")

whitelist = {
     "root": ["C", "F", "G"],
     "chord_type": ["", "m", "dim"],
     "open": [False],
     "starting_fret": [1, 2, 5, 9],
     "include_string": [1, 2],
     "inversion": [1, 2, 3, None],
     "scale": [("C", scales["ionian"])]
 }

filtered_chords = chord_collection.only(whitelist)
print(f"Remaining after whitelist filter: {len(filtered_chords)}")

blacklist = whitelist
filtered_chords = chord_collection.filter_out(blacklist)
print(f"Remaining after blacklist filter: {len(filtered_chords)}")



('A', 'm', finger_ascending=[0, 1, 2, 2, 0, None], starting_fret=1)
('A#', 'm', finger_ascending=[1, 2, 3, 3, 1, None], starting_fret=1)
Collection Size: 132
Collection Size: 328
Remaining after whitelist filter: 1
Remaining after blacklist filter: 327
